# 2 - Why using Machine Learning in Security?

This notebook presents some examples explaining some of the reasons that led to the usage of Artificial Intelligence (in particular, Machine Learning techniques, in this case) in cyber security.

---

## SPAM DETECTION

### Preparation

- Download the 2007 TREC Public Spam Corpus from https://plg.uwaterloo.ca/~gvcormac/treccorpus07/ (255MB)
- Read the "Agreement for use"
- Set up the `datasets` directory
- Untar the corpus in the `datasets` directory

### Starting with the code...
- create constants with the path of the folders containing the data

In [1]:
DATA_DIR = 'datasets/trec07p/data/'
LABELS_FILE = 'datasets/trec07p/full/index'

- import nltk ("Natural Language ToolKit") and download the required packages.
    - the `import` statement lets you gain access to code in another module
    - `nltk` is a suite of libraries and programs for natural language processing (NLP)

In [2]:
import nltk

In [3]:
nltk.download('words')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Error loading words: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>
[nltk_data] Error loading stopwords: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>
[nltk_data] Error loading punkt: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>


False

- define functions to manage the email data (as of now, you don't really have to look at this code, you can run it as black-box)

In [4]:
def flatten_to_string(parts):
    """
    Combine the different parts of the email into a flat list of strings.
    """
    ret = []
    if type(parts) == str:
        ret.append(parts)
    elif type(parts) == list:
        for part in parts:
            ret += flatten_to_string(part)  # Recursion
    elif parts.get_content_type == 'text/plain':
        ret += parts.get_payload()
    return ret

In [5]:
def extract_email_text(path):
    """
    Extract subject and body text from a single email file.
    """
    # Load a single email from an input file
    with open(path, errors='ignore') as f:
        msg = email.message_from_file(f)
    if not msg:
        return ""

    # Read the email subject
    subject = msg['Subject']
    if not subject:
        subject = ""

    # Read the email body
    body = ' '.join(m for m in flatten_to_string(msg.get_payload()) if type(m) == str)
    if not body:
        body = ""

    return subject + ' ' + body

In [6]:
def load(path):
    """
    Process a single email file into stemmed tokens.
    """
    email_text = extract_email_text(path)
    if not email_text:
        return []

    # Tokenize the message
    tokens = nltk.word_tokenize(email_text)

    # Remove punctuation from tokens
    tokens = [i.strip("".join(punctuations)) for i in tokens if i not in punctuations]

    # Remove stopwords and stem tokens
    if len(tokens) > 2:
        return [stemmer.stem(w) for w in tokens if w not in stopwords]
    return []


- import other required modules
    - `string`: module containing common string operations
    - `email`: module for managing email messages
    - `os`: module providing functions to navigate, create, delete and modify files and folders.
    - `pickle`: implements binary protocols for serializing and de-serializing a Python object structure (basically, can be used to store variables)

In [7]:
import string
import email
import os
import pickle

- define a list containing punctuation symbols (cast to `list` is required because `string.punctuation` returns a `str`)

In [8]:
punctuations = list(string.punctuation)

- define the set of stopwords (e.g. "and", "or", etc.)

In [9]:
stopwords = set(nltk.corpus.stopwords.words('english'))

In [10]:
stopwords

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

- define a stemmer to be used for preprocessing text

In [11]:
stemmer = nltk.PorterStemmer()

In [12]:
stemmer.stem("speaking")

'speak'

In [13]:
stemmer.stem('speaks')

'speak'

- collect the labels (i.e. the **real** categories) of the emails from the datasets. 
    - *ham* is mapped to 0 
    - *spam* is mapped to 1

In [14]:
labels = {}
with open(LABELS_FILE) as f:
    for line in f:
        line = line.strip()
        label, key = line.split()
        labels[key.split('/')[-1]] = 1 if label.lower() == 'ham' else 0

In [15]:
type(labels)

dict

#### Q: How many key-value pairs are in the dictionary?

In [16]:
len(labels.keys())

75419

<div class="alert alert-block alert-success">
The keys have to be unique in python dictionaries (i.e. there cannot be equal keys).
Thus, it is enought to print the number of keys to answer the question.
The number of keys is obtained as the length of the object returned by the "keys" method.
    
Below, another possible solution.
</div>

In [17]:
len(labels.items())

75419

#### Q: How many distinct values are in the dictionary?

In [18]:
len(set(list(labels.values())))

2

<div class="alert alert-block alert-success">
labels.values() returns all the values in the dictionary.
We want the distinct values, so we cannot simply look at the length of that (as we did instead for the keys).
    
That's why we create a set (to remove duplicate values) and then look at the length of the set.

The values are only 1 and 0 representing "not spam" and "spam" respectively.
</div>

#### Q: How many distinct keys are in the dictionary?

In [19]:
len(list(labels.keys()))

75419

<div class="alert alert-block alert-success">
Same motivation as above, while printing the number of items.
</div>

- split the corpus in train set and test set

In [20]:
filelist = os.listdir(DATA_DIR)

TRAINING_SET_RATIO = 0.7
X_train = filelist[:int(len(filelist)*TRAINING_SET_RATIO)]
X_test = filelist[int(len(filelist)*TRAINING_SET_RATIO):]

#### Q: Why do we split the data?

<div class="alert alert-block alert-success">
When we train a model, we use the training test but then, in order to measure its actual performance, we use the test set to check whether the model is capable to generalize to previously unseen entries (e.g. in this case, whether it can detect spam emails it was not trained on).
</div>

#### Q: How many elements are in `X_train`?

In [21]:
len(X_train)

52793

<div class="alert alert-block alert-success">
It's a list, we can just print its length
</div>

#### Q: which is the type of the elements?

In [22]:
type(X_train[0])

str

<div class="alert alert-block alert-success">
This isn't really a solution, since you cannot be sure that all the elements are of the same type (remember, in python you can have lists containing elements of different types).
With the cell above you simply know that the first element is a string.
</div>

In [23]:
set([type(x) for x in X_train])

{str}

<div class="alert alert-block alert-success">
This is one actual solution.

It consits in creating a list which is as long as the original one (X_train) and in which each element represents the type of the corresponding element in X_train.
Then, converting this new list into a set we can keep all the distinct types appearing in X_train.
</div>

---

## SPAM DETECTION WITH BLACKLISTED WORDS

- We have to tell the system which words are *spam* and which are *ham*

In [24]:
spam_words = set()
ham_words = set()

In [25]:
if not os.path.exists('blacklist.pkl'):
    for filename in X_train:
        path = os.path.join(DATA_DIR, filename)
        if filename in labels:
            label = labels[filename]
            stems = load(path)
            if not stems:
                continue
            if label == 1:
                ham_words.update(stems)
            elif label == 0:
                spam_words.update(stems)
            else:
                continue
    blacklist = spam_words - ham_words
    pickle.dump(blacklist, open('blacklist.pkl', 'wb'))
else:
    blacklist = pickle.load(open('blacklist.pkl', 'rb') )

print('Blacklist successfully built/loaded')

Blacklist successfully built/loaded


- Let's see some of them...

In [26]:
blacklist

{"style='margin-left:42.0pt",
 'www.massiveservicesmedia.com/i/yz67zf/ewovpe36/wuror_4131/tecob_1.gif',
 'p=8bm109-1',
 'has/been/steadily/extending/its/range./sightings/now/commonly/occur/in/california',
 'logitech',
 'ϼƻִ취',
 'steam-pip',
 'sqnawlvhic8gq2hpbmegq2hlbmdoywkgsw50zxjuyxrpb25hbcbub3lzicygr2lmdhmgrmfpcian',
 'hilario',
 'ι̬',
 '7998',
 'www.2000laneswatch.com/i/8ztvno/mctnfceq/dokix_4935/mufey_10.bmp',
 'id=lw_1182947048_2',
 'mebut',
 'stobhd.guideleper.info',
 'dutch//mot//and/german//motte//all/meaning//moth',
 'unalloy',
 '1875',
 'incubando',
 'mso-list-id:1881163078',
 'viagra50mg',
 'www.picrangeinternet.com/i/oi6d2t/wpvx8ah7/somum_6396/nocuz_11.jpg',
 'fashuntri.com/t/kg5gw0skt/13158',
 'alabama///br//sweet//home/alabama//unoffici',
 'comprehensivecoaching.com',
 '1993/ref',
 '104.1',
 'rubrip',
 'cypselid',
 'bersichtschwarz',
 'gatepost',
 'rope/mak',
 'forpenisenlarg',
 'that=20',
 'jethro',
 'elekonich',
 'latchkey',
 't0nluvvpveu+dqogicagicagidxuqujmrsbjzwxsu3

#### Q: How many elements in the set?

In [27]:
len(blacklist)

97939

#### Q: Is "spam" in the set?

In [28]:
'spam' in blacklist

False

<div class="alert alert-block alert-success">
Many ways to do this (e.g. see cell below), but this is the fastest.
</div>

In [29]:
found = False
for x in blacklist:
    if x == 'spam':
        found = True
if found:
    print(True)
else:
    print(False)

False


#### Q: is "fibonacci" in the set?

In [30]:
'fibonacci' in blacklist

True

#### Q: How long is the longest word in the set?

In [31]:
max([len(x) for x in blacklist])

1984

<div class="alert alert-block alert-success">
This is similar to the example in which we had to find the type of all the elements in X_train: we create a list in which each element represents the lenght of one of the elements in blacklist, then we get the max from this new list.
</div>

#### Q: Which word is it?

In [32]:
max_len = max([len(x) for x in blacklist])
for word in blacklist:
    if len(word) == max_len:
        print(word)

geobox/river//////////name/section///////////name///dnieper/native_name///other_name///other_name1///other_name2////////////map/section///////////map///dnipro/basin/river/town/international.png/map_size///map_caption///the///dnieper/s/drainage/basin//////////country/etc.///////////country///russia/country1///belarus/country2///ukraine/region///region1///city/////dorogobuzh/city1///smolensk/city2///mahilyow/city3///kiev/city4///cherkasy///city5///dnipropetrovsk//////////geography///////////length///2290/length_imperial///watershed///516300/watershed_imperial/////discharge_location///kherson/discharge_average///1670///discharge_average_imperial////////////source///////////source_name///glaciers/source_location///valdai/hills/source_country///russia///source_lat_d///55/source_lat_m///52/source_lat_s///00/source_lat_ns///n///source_long_d///33/source_long_m///41/source_long_s///00/source_long_ew///e///source_elevation///220/source_elevation_imperial////////////mouth///////////mouth_name///

<div class="alert alert-block alert-success">
I scan all the blacklisted words and print the ones whose length is equal to the max length.
</div>

- But these are not really "words"... Lt's look only at the actual words

In [33]:
from nltk.corpus import words
word_set = set(words.words())
word_blacklist = word_set.intersection(blacklist)

In [34]:
word_blacklist

{'herbalist',
 'bemask',
 'occultist',
 'sandalwood',
 'manipular',
 'pantaloon',
 'krypton',
 'munj',
 'downbear',
 'mainsail',
 'circumpolar',
 'transept',
 'dhai',
 'flung',
 'tapia',
 'despond',
 'vodka',
 'varanid',
 'quorum',
 'whitebark',
 'inearth',
 'bullhead',
 'dern',
 'hellish',
 'smew',
 'dactyl',
 'dha',
 'edict',
 'accrual',
 'strangler',
 'rampart',
 'rattail',
 'ironclad',
 'auk',
 'pecker',
 'shrew',
 'ka',
 'gatepost',
 'atlas',
 'shoo',
 'wallboard',
 'flavo',
 'duller',
 'latchkey',
 'alarum',
 'refound',
 'jamb',
 'starboard',
 'shedder',
 'duet',
 'appet',
 'outspread',
 'breadroot',
 'productid',
 'chaplain',
 'teapot',
 'yip',
 'reservoir',
 'druggist',
 'chambermaid',
 'humanist',
 'gambit',
 'catchup',
 'gynoecium',
 'tinct',
 'mincemeat',
 'caudal',
 'archaic',
 'carob',
 'prion',
 'yin',
 'sneer',
 'regia',
 'headstrong',
 'stam',
 'unaccustom',
 'assi',
 'sitch',
 'smeek',
 'verdant',
 'cebid',
 'dockyard',
 'silverback',
 'lintel',
 'chaff',
 'eyelash',
 

#### Q: How many elements in the set?

In [35]:
len(word_blacklist)

3160

#### Q: How long is the longest word in the set?

In [36]:
max([len(x) for x in word_blacklist])

16

#### Q: Which word is it?

In [37]:
max_len = max([len(x) for x in word_blacklist])
for word in word_blacklist:
    if len(word) == max_len:
        print(word)

microlepidoptera
macrolepidoptera


## Let's use this model!

#### Metrics for classification

- Building blocks for the metrics
    - TP = True Positive
    - TN = True Negative
    - FP = False Positive
    - FN = False Negative


- The actual metrics:
    - $\text{accuracy} = \frac{TP + TN}{TP + TN + FP + FN}$
    - $\text{precision} = \frac{TP}{TP + FP}$; (a.k.a. positive predictive value)
    - $\text{recall} = \frac{TP}{TP + FN}$; (a.k.a. sensitivity, hit rate, true positive rate)

In [38]:
# this cell might take quite some time
fp = 0
tp = 0
fn = 0
tn = 0

for filename in X_test:
    path = os.path.join(DATA_DIR, filename)
    if filename in labels:
        label = labels[filename]
        stems = load(path)
        if not stems:
            continue
        stems_set = set(stems)
        if stems_set & blacklist:  # INTERSECTION BETWEEN SETS
            if label == 1:
                fp = fp + 1
            else:
                tp = tp + 1
        else:
            if label == 1:
                tn = tn + 1
            else:
                fn = fn + 1

In [39]:
print("TN %d" % tn)
print("FP %d" % fp)
print("FN %d" % fn)
print("TP %d" % tp)

TN 7352
FP 211
FN 5284
TP 7996


#### Q: Print the TP, FP, TN, FN as percentage (or fraction).

In [40]:
count = tn + tp + fn + fp
print("TN %.2f" % (tn/count))
print("FP %.2f" % (fp/count))
print("FN %.2f" % (fn/count))
print("TP %.2f" % (tp/count))

TN 0.35
FP 0.01
FN 0.25
TP 0.38


In [41]:
print("Confusion matrix:\n")
print("| TN %.2f | FP %.2f |" % (tn/count, fp/count))
print("| FN %.2f | TP %.2f |" % (fn/count, tp/count))

Confusion matrix:

| TN 0.35 | FP 0.01 |
| FN 0.25 | TP 0.38 |


In [42]:
print("Accuracy: %.5f" % ((tp+tn)/count))
print("Precision: %.5f" % (tp/(tp+fp)))
print("Recall: %.5f" % (tp/(tp+fn)))

Accuracy: 0.73636
Precision: 0.97429
Recall: 0.60211


---

## Logistic Regression

Let's try now with a different model.

In [43]:
def read_email_files():
    X = []
    y = [] 
    for i in range(len(labels)):
        filename = 'inmail.' + str(i+1)
        email_str = extract_email_text(os.path.join(DATA_DIR, filename))
        X.append(email_str)
        y.append(labels[filename])
    return X, y

- "Read" the emails

In [44]:
X, y = read_email_files()

- split in train set and test set

In [45]:
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    X, y, range(len(y)), train_size=TRAINING_SET_RATIO, random_state=2
)

/home/luca/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


#### Q: How does the input data (i.e. X) look like? (types, content, ...)

In [46]:
type(x for x in X_train)

generator

<div class="alert alert-block alert-success">
A generator can be thought of as a list that can iterated on (e.g. with a for loop) only once.
</div>

In [47]:
X_train[3]

'The Best for Your Health! '

#### Q: and the target label?

In [48]:
y_test[0]

1

In [49]:
set(y_test)

{0, 1}

- As input, we have strings (emails). We **have** to convert them into numbers.

#### Q: Any ideas on how to convert strings into numbers? (hint: think about the stemmer we have seen...)

- Tons of approaches are possible, here we use `CountVectorizer`

<div class="alert alert-block alert-success">
CountVectorizer converts each element of the original generator in a new array.
In the originar X_train we had one entry for each email of the training set, after CountVerctorizer we still have one element for each email of the training set, but they are arrays instead of strings.
the length of each array is equal to the number of words in the input corpus (i.e. the training emails) and each cell represents the number of occurrences of the corresponding word in the corresponding email.
</div>

In [50]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_train_vector = vectorizer.fit_transform(X_train)
X_test_vector = vectorizer.transform(X_test)

#### Q: Let's look at the input data now. How is it different from before?

In [51]:
type(x for x in X_train_vector)

generator

In [52]:
X_train_vector[3]

<1x238197 sparse matrix of type '<class 'numpy.int64'>'
	with 5 stored elements in Compressed Sparse Row format>

- let's define and train the model!

In [53]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train_vector, y_train)

/home/luca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/luca/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

- and check its predictions

In [54]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score
)

y_pred = clf.predict(X_test_vector)

print("Accuracy:  %.5f" % accuracy_score(y_test, y_pred))
print("Precision: %.5f" % precision_score(y_test, y_pred))
print("Recall:    %.5f" % recall_score(y_test, y_pred))

Accuracy:  0.97291
Precision: 0.99294
Recall:    0.92583


- You can play with the parameters of the model, looking for the best configuration (there is an approach in order to do this in an "organized" way). We will see that in the future.

In [55]:
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial', max_iter=250)
clf.fit(X_train_vector, y_train)

y_pred = clf.predict(X_test_vector)

print("Accuracy:  %.5f" % accuracy_score(y_test, y_pred))
print("Precision: %.5f" % precision_score(y_test, y_pred))
print("Recall:    %.5f" % recall_score(y_test, y_pred))

Accuracy:  0.98643
Precision: 0.99017
Recall:    0.96917


/home/luca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


<div class="alert alert-block alert-info">
In some cases, it is not trivial to understand which is the best model, while performing a comparison. 
For instance, how should we behave when we have several performance metrics and they do not agree on the best model?
The choice depends on the requirements of the specific task at hand: e.g. if we want to detect all the spam emails even if this causes a farily big number of false positives, we will pick a model with high recall.
</div>

---

## Decision Tree Classifier

In [56]:
from sklearn.tree import DecisionTreeClassifier

# Define the model
clf = DecisionTreeClassifier()

# Train the model (this might take quite some time)
clf.fit(X_train_vector, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [57]:
y_pred = clf.predict(X_test_vector)

print("Accuracy:  %.5f" % accuracy_score(y_test, y_pred))
print("Precision: %.5f" % precision_score(y_test, y_pred))
print("Recall:    %.5f" % recall_score(y_test, y_pred))

Accuracy:  0.98391
Precision: 0.98058
Recall:    0.97128


- Let's try different parameters!

In [58]:
# Define the model
clf = DecisionTreeClassifier(max_leaf_nodes=2)

# Train the model (this might take quite some time)
clf.fit(X_train_vector, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=2, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [59]:
y_pred = clf.predict(X_test_vector)

print("Accuracy:  %.5f" % accuracy_score(y_test, y_pred))
print("Precision: %.5f" % precision_score(y_test, y_pred))
print("Recall:    %.5f" % recall_score(y_test, y_pred))

Accuracy:  0.86317
Precision: 0.92704
Recall:    0.64273


- that's not very good... let's check how it performed on the training data

In [60]:
y_pred = clf.predict(X_train_vector)

print("Accuracy:  %.5f" % accuracy_score(y_train, y_pred))
print("Precision: %.5f" % precision_score(y_train, y_pred))
print("Recall:    %.5f" % recall_score(y_train, y_pred))

Accuracy:  0.86350
Precision: 0.92756
Recall:    0.64133


- if the performance is poor both on the training set and the test set, it is a case of **UNDERFITTING**.
- if the performance is poor on the test set but good on the training set, it is a case of **OVERFITTING**. (Basically, the model is not able to generalize).

---

## Random Forest Classifier

#### Q: Now it's your turn! Try to build a Random Forest classifier.

In [61]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100)

clf.fit(X_train_vector, y_train)

y_pred = clf.predict(X_test_vector)

print("Accuracy:  %.5f" % accuracy_score(y_test, y_pred))
print("Precision: %.5f" % precision_score(y_test, y_pred))
print("Recall:    %.5f" % recall_score(y_test, y_pred))

Accuracy:  0.98709
Precision: 0.99244
Recall:    0.96891


---